In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-26 02:45:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.49MB/s    in 0.2s    

2022-02-26 02:45:45 (4.49 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-2").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

**Load Amazon Data into Spark DataFrame**

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
music_df.count()

4751577

In [7]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = music_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3LI5TRP3YIDQL|   10140119|B00TXH4OLC|     384427924| 2015-08-31|
|R3LGC3EKEG84PX|   27664622|B00B6QXN6U|     831769051| 2015-08-31|
| R9PYL3OYH55QY|   45946560|B001GCZXW6|      14067376| 2015-08-31|
|R3PWBAWUS4NT0Q|   15146326|B000003EK6|     566295619| 2015-08-31|
|R15LYP3O51UU9E|   16794688|B00N1F0BKK|     210426072| 2015-08-31|
|R1AD7L0CC3DSRI|   32203364|B00V7KAO7Q|     360249504| 2015-08-31|
|R32FE8Y45QV434|    1194276|B000094Q4P|      20972283| 2015-08-31|
|R3NM4MZ4XWL43Q|   45813052|B00JMK0P1I|     378049560| 2015-08-31|
|R3H4FXX6Q7I37D|   12795687|B008OW1S3O|     141620702| 2015-08-31|
|R30L5PET7LFFDC|   36673840|B00VI2L3L4|     920557135| 2015-08-31|
|  REFRE1LEKLAF|   49453576|B0000041EV|     683128964| 2015-08-31|
|R3JTJ5EQN74E9H|    3285047|B00005YW4H|     268461622| 2015-08

In [11]:
review_id_df.count()

4751577

In [8]:
# drop the duplicates
products_df = music_df.select(["product_id", "product_title"]).drop_duplicates()

In [9]:
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001FOJ76S|         The Promise|
|B000NJWSE4|                Free|
|B00002CF4Z|The Best of Steve...|
|B000CQM4T2|        Live Trucker|
|B000002QG8|            Float On|
|B003JYOFIW|Something for Eve...|
|B004MY64NM|             Manhole|
|B007I8TCD8|           Do Things|
|B001CY2ELQ|               Lenka|
|B00BQ1DBH6|The Best From The...|
|B00VXGTJMU|   Django and Jimmie|
|B00MI9OZ0W|BAYONETTA 2 Origi...|
|B003DC881A|   Chamberlain Waits|
|B000003MZG|            Africano|
|B00005QIUW|Christmas Through...|
|B000WSTAEC|Haendel: La Passi...|
|B002PXJZIS|Dont Stop Believi...|
|B00U646N7E|Directions Home (...|
|B0007MR1L2|Hummel: Septet, O...|
|B00AK77XUS|     The Flower Lane|
+----------+--------------------+
only showing top 20 rows



In [12]:
products_df.count()

782353

In [10]:
# group customer id by customer count
customer_df=music_df.groupBy("customer_id").count().withColumnRenamed("count","customer_count")
customer_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     651796|             2|
|   44979559|             1|
|   45632184|             1|
|   15232809|             1|
|   17067926|             1|
|   26163929|             1|
|   20587872|             2|
|   14230926|             3|
|   52081222|             2|
|   13382150|             1|
+-----------+--------------+
only showing top 10 rows



In [13]:
customer_df.count()

1940732

In [14]:
# select columns from vine_df
vine_df=music_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|
|R3LGC3EKEG84PX|          5|            0|          0|   N|
| R9PYL3OYH55QY|          5|            0|          1|   N|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|
|R15LYP3O51UU9E|          5|            0|          0|   N|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|
|R32FE8Y45QV434|          5|            0|          0|   N|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|
|R30L5PET7LFFDC|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [15]:
vine_df.count()

4751577

**Write Data to AWS RDS Instance**

In [16]:
# set up properties to connect to AWS Postgres
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cykufjcvydqa.us-east-2.rds.amazonaws.com:5432/ProductReviewsDB"
config = {"user":"postgres", "password": "", "driver":"org.postgresql.Driver"}

In [17]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode="overwrite", properties=config)

In [21]:
# Write customers_df to table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode="overwrite", properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)